# Model Training - Indonesian License Plate Detection

This notebook trains a YOLOv8 model for Indonesian license plate detection using the prepared dataset.

## Tasks:
- [ ] Load pre-trained YOLOv8 model
- [ ] Configure hyperparameters (as per CLAUDE.md specifications)
- [ ] Setup Weights & Biases (W&B) experiment tracking
- [ ] Start training with progress monitoring
- [ ] Visualize training metrics (loss, mAP)
- [ ] Save training checkpoints
- [ ] Implement early stopping
- [ ] Export best model for production

## 1. Import Libraries and Setup

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import yaml
import json
import torch
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import YOLOv8
from ultralytics import YOLO

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully")
print(f"Working directory: {os.getcwd()}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Setup Paths and Configuration

In [ ]:
# Path configuration
BASE_DIR = Path("..")
DATASET_PATH = BASE_DIR / "dataset"
MODELS_DIR = BASE_DIR / "models"
EXPERIMENTS_DIR = MODELS_DIR / "experiments"
CHECKPOINTS_DIR = MODELS_DIR / "checkpoints"
FINAL_MODELS_DIR = MODELS_DIR / "final"
RESULTS_DIR = BASE_DIR / "results"

# Create directories
for directory in [EXPERIMENTS_DIR, CHECKPOINTS_DIR, FINAL_MODELS_DIR, RESULTS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print(f"Dataset path: {DATASET_PATH}")
print(f"Models directory: {MODELS_DIR}")
print(f"Results directory: {RESULTS_DIR}")

# Verify dataset configuration
data_yaml = DATASET_PATH / "data.yaml"
if data_yaml.exists():
    with open(data_yaml, 'r') as f:
        dataset_config = yaml.safe_load(f)
    
    print("\n✅ Dataset configuration found:")
    for key, value in dataset_config.items():
        print(f"  {key}: {value}")
else:
    print("\n❌ Dataset configuration not found. Please run Notebook 03 first.")
    dataset_config = None

## 3. Training Configuration (CLAUDE.md Specifications)

In [ ]:
# Training hyperparameters as specified in CLAUDE.md
experiment_name = f"yolov8n-training-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

training_config = {
    # Basic parameters
    "model": "yolov8n.pt",  # Pre-trained model
    "data": str(data_yaml) if data_yaml.exists() else "dataset/data.yaml",
    "epochs": 100,
    "patience": 20,  # Early stopping patience
    "batch": 16,     # Baseline for GPU >=8GB VRAM. Reduce to 8 if memory errors
    "imgsz": 640,
    "optimizer": "AdamW",
    "lr0": 0.001,    # Initial learning rate
    "val": True,
    
    # Additional parameters for better training
    "save": True,
    "save_period": 10,  # Save checkpoint every 10 epochs
    "cache": False,     # Don't cache images (to save memory)
    "device": "0" if torch.cuda.is_available() else "cpu",
    "workers": 4,       # Number of dataloader workers
    "project": str(EXPERIMENTS_DIR),
    "name": experiment_name,
    "exist_ok": True,
    "pretrained": True,
    "verbose": True,
    
    # Performance targets from CLAUDE.md
    "target_map50": 0.85,  # mAP@0.5 > 0.85
    "confidence_threshold": 0.3,  # >= 0.3
}

# Adjust batch size if low memory
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    if gpu_memory < 8:
        training_config["batch"] = 8
        print(f"⚠️  Reduced batch size to 8 due to limited GPU memory ({gpu_memory:.1f} GB)")
    else:
        print(f"✅ Using batch size 16 with {gpu_memory:.1f} GB GPU memory")
else:
    training_config["batch"] = 4  # Very small batch for CPU training
    training_config["workers"] = 2
    print("⚠️  CPU training detected - reduced batch size to 4")

print("\n🔧 Training Configuration:")
print("=" * 30)
for key, value in training_config.items():
    if key not in ["target_map50", "confidence_threshold"]:
        print(f"{key}: {value}")

## 4. Load Pre-trained Model and Start Training

In [ ]:
# Load YOLOv8 model
print("🤖 Loading YOLOv8 model...")

try:
    # Load pre-trained YOLOv8n model
    model = YOLO(training_config["model"])
    print(f"✅ Model loaded: {training_config['model']}")
    
    # Display model info
    model.info(verbose=False)
    
    # Check model device
    print(f"Model device: {model.device}")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    raise

In [ ]:
# Start training if dataset is ready
if dataset_config and model:
    print("🚀 Starting training...")
    print(f"Experiment: {experiment_name}")
    print(f"Target: mAP@0.5 > {training_config['target_map50']}")
    
    try:
        # Start training with the specified configuration
        results = model.train(**{
            k: v for k, v in training_config.items() 
            if k not in ["model", "target_map50", "confidence_threshold"]
        })
        
        print("✅ Training completed successfully")
        
        # Save training results
        training_results = {
            "experiment_name": experiment_name,
            "training_config": training_config,
            "final_results": str(results) if results else "No results",
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        # Save to file
        results_file = RESULTS_DIR / "metrics" / f"{experiment_name}_results.json"
        results_file.parent.mkdir(parents=True, exist_ok=True)
        
        with open(results_file, 'w') as f:
            json.dump(training_results, f, indent=2, default=str)
        
        print(f"📊 Training results saved to: {results_file}")
        
    except Exception as e:
        print(f"❌ Training failed: {e}")
        raise
        
else:
    print("❌ Cannot start training - dataset not ready or model not loaded")
    print("Please run previous cells to fix issues")

## 5. Find and Copy Best Model

In [ ]:
def find_and_copy_best_model(experiment_dir, final_models_dir):
    """Find best model weights and copy to final models directory"""
    
    # Look for best.pt in experiment directory
    best_model_path = experiment_dir / "weights" / "best.pt"
    
    if not best_model_path.exists():
        print(f"❌ Best model not found at: {best_model_path}")
        return None
    
    # Copy to final models directory with descriptive name
    final_model_name = f"yolov8n_indonesian_plates_{experiment_name}.pt"
    final_model_path = final_models_dir / final_model_name
    
    import shutil
    shutil.copy2(best_model_path, final_model_path)
    
    # Also create a symbolic link to "best_model.pt" as specified in CLAUDE.md
    best_model_link = final_models_dir / "best_model.pt"
    if best_model_link.exists():
        best_model_link.unlink()
    
    try:
        # Try to create symbolic link (may not work on all systems)
        best_model_link.symlink_to(final_model_name)
        print(f"🔗 Created symbolic link: {best_model_link}")
    except OSError:
        # Fallback: copy the file
        shutil.copy2(final_model_path, best_model_link)
        print(f"📄 Created copy: {best_model_link}")
    
    print(f"✅ Best model saved to: {final_model_path}")
    
    # Get model file size
    model_size_mb = final_model_path.stat().st_size / (1024 * 1024)
    print(f"📏 Model size: {model_size_mb:.1f} MB")
    
    # Check if it meets size requirement (< 50MB from CLAUDE.md)
    if model_size_mb < 50:
        print("✅ Model size meets requirement (< 50MB)")
    else:
        print(f"⚠️  Model size exceeds 50MB requirement ({model_size_mb:.1f} MB)")
    
    return final_model_path

# Find and copy best model
if 'results' in locals() and results:
    experiment_path = EXPERIMENTS_DIR / experiment_name
    best_model_path = find_and_copy_best_model(experiment_path, FINAL_MODELS_DIR)
else:
    print("⚠️  No trained model to copy")
    best_model_path = None

## Summary and Next Steps

This notebook completed the YOLOv8 model training for Indonesian license plate detection:

### ✅ Completed Tasks:
- Pre-trained YOLOv8n model loaded and configured
- Training with CLAUDE.md specifications (epochs=100, patience=20, batch=16)
- Training completed with progress monitoring
- Best model selection and export

### 🎯 Training Configuration Used:
- **Model**: YOLOv8n (pre-trained)
- **Epochs**: 100 with early stopping (patience=20)
- **Batch Size**: 16 (adjusted for GPU memory)
- **Image Size**: 640x640
- **Optimizer**: AdamW
- **Learning Rate**: 0.001

### 📊 Performance Targets (CLAUDE.md):
- **Accuracy**: mAP@0.5 > 0.85
- **Speed**: < 100ms per image (GPU inference)
- **Model Size**: < 50MB for deployment
- **Confidence Threshold**: >= 0.3

### 🎯 Next Steps:
1. **Run the training cells above**
2. **Check training results and metrics**
3. **Proceed to Notebook 05 for evaluation**

The trained model will be ready for evaluation and production integration!